# System_simulation


In [ ]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
from extended_kalman_filter import ExtendedKalmanFilter

In [ ]:
def plot_util(_plt):
    _plt.legend()
    _plt.grid()
    fig = _plt.gcf();
    fig.set_size_inches(18.5, 5, forward=True)
    _plt.show()

## Load the pre-recorded data

In [ ]:
df_loaded = pd.read_csv('logged_data_1.csv', index_col=0)
print('Columns:', [i for i in df_loaded.columns])

phi_yaw_arr = df_loaded.iloc[:,0].values
phi_pitch_arr = df_loaded.iloc[:,1].values
time_arr = df_loaded.iloc[:,2].values

## Sample Kalman Filters

In [ ]:
EKF_1 = ExtendedKalmanFilter(Ts=0.05, Q=0.1,   R=0.1, a=1)
EKF_2 = ExtendedKalmanFilter(Ts=0.05, Q=0.01,  R=0.1, a=1)
EKF_3 = ExtendedKalmanFilter(Ts=0.05, Q=1e-3, R=0.1, a=1)
EKF_4 = ExtendedKalmanFilter(Ts=0.05, Q=1e-3, R=1, a=1)

EKFs =  [EKF_1, EKF_2, EKF_3, EKF_4]
labels= ['Q=0.1, R=0.1, a=1']

In [ ]:
results = []
prev_measurement = 0

for EKF in EKFs:
    temp_results = []
    for measurement in phi_yaw_arr:
        EKF.predict()
        
        if measurement != prev_measurement:
            EKF.update(measurement)
            prev_measurement = measurement
        
        temp_results.append(EKF.x.item(0))
    
    results.append(temp_results)

In [ ]:
plt.plot(time_arr, phi_yaw_arr, label='Raw NN estimate of yaw [deg]')
for i, result in enumerate(results):
    plt.plot(time_arr, result, label='EKF_%i' % (i+1))
plt.title('Comparison of EKFs with different parameters for Q, R and a')
plot_util(plt)

In [ ]:
# plt.plot(time_arr, EKF_yaw_arr, label='EKF estimate of yaw [deg]')
plt.plot(time_arr, phi_yaw_arr, label='Raw NN estimate of yaw [deg]')
plot_util(plt)

In [ ]:
# plt.plot(time_arr, EKF_pitch_arr, label='EKF estimate of pitch [deg]')
plt.plot(time_arr, phi_pitch_arr, label='Raw NN estimate of pitch [deg]')
plot_util(plt)

In [ ]:
loop_times_ms = [(t-t_)*1e3 for t_,t in zip(time_arr[0:-1], time_arr[1:])]
plt.stem(time_arr[:-1], loop_times_ms, label='loop times [ms]')
plot_util(plt)

In [ ]:
from ipywidgets import interact

In [ ]:
def sim_and_plot_EKF(Q, R, a):
    EKF = ExtendedKalmanFilter(Ts=0.05, Q=Q, R=R, a=a)
    print('Q = %f, R = %f' % (Q, R))
    data = phi_yaw_arr
#     data = phi_pitch_arr
    prev_measurement = 0
    results = []
    predict_ahead_results = []
    
    for measurement in data:
        EKF.predict()
        
        if measurement != prev_measurement:
            EKF.update(measurement)
            prev_measurement = measurement
        
        results.append(EKF.x.item(0))
        predict_ahead_results.append(EKF.predict_ahead(3)) # 3*Ts = 150 ms

    plt.plot(time_arr, data, label='Raw NN estimate [deg]')
    plt.plot(time_arr, results, label='EKF estimate [deg]')
    plt.plot(time_arr, predict_ahead_results, label='EKF estimate predicting 150 ms into future [deg]')
    plot_util(plt)

In [ ]:
interact(sim_and_plot_EKF, Q=(1e-4, 1, 1e-4), R=(1e-3, 1e-2, 1e-3), a=0)#(0.9, 1.0, 0.01));

In [ ]:
# good values: Q = 0.5, R = 0.005, a = 0.94